In [ ]:
def validation(valid_dataset, gen_model, dis, losses):
    
    gen_loss = 0
    dis_loss = 0
    total_psnr = 0
    total_ssim = 0
    num_batches = 0
    total_samples = 0

    for hr, lr in valid_dataset:
           
        batch_size = tf.shape(hr)[0]
        gen_output = gen_model(lr, training=False)
        generator_loss = losses(dis, lr, gen_output, hr)
        
        dis_real_output = dis([hr, lr], training=False)
        dis_fake_output = dis([gen_output, lr], training=False)
        discriminator_loss = dis_loss(dis_real_output, dis_fake_output)
        
        # Calculate metrics
        psnr_value = psnr(hr, gen_output)
        ssim_value = ssim(hr, gen_output)

        # Weighted accumulation (by batch size)
        gen_loss += generator_loss.numpy() * batch_size
        dis_loss += discriminator_loss.numpy() * batch_size
        total_psnr += psnr_value.numpy() * batch_size
        total_ssim += ssim_value.numpy() * batch_size
        total_samples += batch_size
        num_batches += 1
    
    # Calculate proper weighted averages
    avg_gen_loss = gen_loss / max(1, total_samples)
    avg_dis_loss = dis_loss / max(1, total_samples)
    avg_psnr = total_psnr / max(1, total_samples)
    avg_ssim = total_ssim / max(1, total_samples)

    return avg_gen_loss, avg_dis_loss, avg_psnr, avg_ssim
